In [163]:
import matplotlib
matplotlib.use('Agg')  # Backend non-interactif

import matplotlib.pyplot as plt
import os
import glob
import numpy as np
import astropy.units as u
import random
import pandas as pd

from astropy.wcs import WCS
from textwrap import fill
from astropy.table import Table, vstack, Column
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle
from astropy.io import fits
from astropy.coordinates import search_around_sky
from astropy.visualization import simple_norm
from astropy.table import join, hstack

from constantes import LIM_FLUX_CLUSTER, LIM_FLUX_AGN, SEARCH_RADIUS_CLUSTER, SEARCH_RADIUS_AGN, EXT_LIKE_C1, EXT_LIKE_C2, EXT_C1_C2, PNT_DET_ML_SPURIOUS, EXT_LIKE_SPURIOUS
from constantes import print_parameters

print_parameters()

╭─────────────────────────────────────────────╮
│                  PARAMÈTRES                 │
├─────────────────────────────────────────────┤
│ LIM_FLUX_CLUSTER      : 3.00e-15 erg/cm²/s 
│ LIM_FLUX_AGN          : 3.00e-15 erg/cm²/s 
│ SEARCH_RADIUS_CLUSTER : 30.00    arcsec    
│ SEARCH_RADIUS_AGN     : 10.00    arcsec    
│ EXT_LIKE_C1           : 33       
│ EXT_LIKE_C2           : 15       
│ EXT_C1_C2             : 5        
│ EXT_LIKE_C1_new       : 80       
│ EXT_LIKE_C2_new       : 35       
│ EXT_C1_C2_new         : 13       
│ window_size           : 3.0      arcmin    
│ PNT_DET_ML_SPURIOUS   : 20       
│ EXT_LIKE_SPURIOUS     : 15       
╰─────────────────────────────────────────────╯


# **Suppression des doublons Xamin sur les zones de recouvrement**

In [164]:
suppression_doublons_Xamin = True

if(suppression_doublons_Xamin):

    catalog_path_aftXamin = os.path.expanduser('~/Documents/TransformerProject/data/catalogs/Data_10ks/onlyMOSPN/merged_catalog.fits')
    data_Xamin = Table.read(catalog_path_aftXamin)
    data_Xamin['new_ID'] = np.arange(len(data_Xamin))

    RADIUS_XAMIN = 5  #arcsec

    coords = SkyCoord(ra=data_Xamin['PNT_RA'], dec=data_Xamin['PNT_DEC'], unit='deg')

    #Calcul de ttes les paires a moins de RADIUS_XAMIN
    idx1, idx2, sep2d, _ = coords.search_around_sky(coords, seplimit= RADIUS_XAMIN* u.arcsec)

    #Filtrage des paires distinctes
    mask = idx1 < idx2
    idx1 = idx1[mask]
    idx2 = idx2[mask]

    to_remove = set()
    for i, j in zip(idx1, idx2):
        id_i = data_Xamin['new_ID'][i]
        id_j = data_Xamin['new_ID'][j]
        to_remove.add(i if id_i > id_j else j)

    #Suppression des lignes
    mask_keep = np.ones(len(data_Xamin), dtype = bool)
    mask_keep[list(to_remove)] = False
    data_Xamin_cleaned = data_Xamin[mask_keep]

    data_Xamin_cleaned.remove_column('new_ID')

    # Vu qu'on a supprime des sources la numerotation de ID_Xamin n'etait plus continue
    data_Xamin_cleaned.remove_column('ID_Xamin')
    data_Xamin_cleaned['ID_Xamin'] = np.arange(len(data_Xamin_cleaned))

    print(f"Nombre de sources Xamin avant la suppresion des doublons: {len(data_Xamin)}")
    print(f"Nombre de sources Xamin apres la suppresion des doublons: {len(data_Xamin_cleaned)}")

    input_dir = "~/Documents/TransformerProject/data/catalogs/Data_10ks/onlyMOSPN/"
    output_path = os.path.join(input_dir, "merged_catalog_cleaned.fits") 
    data_Xamin_cleaned.write(output_path, format='fits', overwrite=True)

    print(f"Catalogue Xamin sauvegardé dans : {output_path}")

Nombre de sources Xamin avant la suppresion des doublons: 21152
Nombre de sources Xamin apres la suppresion des doublons: 17752
Catalogue Xamin sauvegardé dans : ~/Documents/TransformerProject/data/catalogs/Data_10ks/onlyMOSPN/merged_catalog_cleaned.fits


# **Noms des chemins vers les catalogues**

In [165]:
activate_cut_Xamin_with_min_40photons = False

In [166]:
catalog_path_AMAS  = os.path.expanduser('~/Documents/TransformerProject/data/catalogs/XFSII_25_sx_p18_b05rc02_output.csv')
catalog_path_Xamin = os.path.expanduser('~/Documents/TransformerProject/data/catalogs/Data_10ks/onlyMOSPN/merged_catalog_cleaned.fits')
catalog_path_AGN   = os.path.expanduser('~/Documents/TransformerProject/data/Flagship_data/FS2_MAMBO_AGN.fits')

data_Xamin      = Table.read(catalog_path_Xamin)
data_input_AMAS = Table.read(catalog_path_AMAS)
data_input_AGN  = Table.read(catalog_path_AGN)

data_Xamin['new_ID'] = np.arange(len(data_Xamin))

if(activate_cut_Xamin_with_min_40photons):
    data_Xamin['Ntot'] = data_Xamin['INST0_EXP'] * data_Xamin['PNT_RATE_MOS'] + data_Xamin['INST1_EXP'] * data_Xamin['PNT_RATE_PN']
    mask_Xamin_sup_40ph = (data_Xamin['Ntot'] > 40)
    data_Xamin = data_Xamin[mask_Xamin_sup_40ph]

new_column_names = ['ID', 'R.A.', 'Dec', 'px', 'yx', 'm200', 'Tsl', 'Lx_soft', 'flux', 'flux_ABS', 'r500', 'z']

current_cols = data_input_AMAS.colnames
print("Colonnes actuelles:")
print(current_cols)

for i in range(1, 13): # Renommer uniquement les colonnes 2 à 13
    data_input_AMAS.rename_column(current_cols[i], new_column_names[i-1])

print("\nColonnes après renommage:")
print(data_input_AMAS.colnames)

Colonnes actuelles:
['3', '513636967', '148.852551198248', '3.66324683120872', '3747.87232503616', '3475.58376807197', '7621999900000', '0.56', '4.07E+41', '3.21E-17', '2.40E-17', '11.52', '0.8423']

Colonnes après renommage:
['3', 'ID', 'R.A.', 'Dec', 'px', 'yx', 'm200', 'Tsl', 'Lx_soft', 'flux', 'flux_ABS', 'r500', 'z']


In [167]:
data_input_AMAS[0]

3,ID,R.A.,Dec,px,yx,m200,Tsl,Lx_soft,flux,flux_ABS,r500,z
int64,int64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64
2,543737908,151.22570493977,6.13184952042855,7150.37004691358,7040.54097352349,19800000600000,0.98,1.53e+42,9.79e-17,7.71e-17,13.2,1.0714


In [168]:
data_input_AGN[0]

id,ra,dec,z,m,passive,agn_type,Lx_h,Lx_s,Fx_s_abs,Fx_s_G14,NH,Mbh
int64,float32,float32,float32,float32,int32,int32,float32,float32,float32,float32,float32,float32
35353500011970000,150.84576,1.9766083,0.003286787,7.7614284,0,2,39.93776,39.478336,4.417671e-15,1.2650335e-13,22.95448,0.0150628565


# **Suppresion des fausses sources**

In [169]:
suppr_spurious = False

if (suppr_spurious):
    mask_spurious = np.logical_or(data_Xamin['PNT_DET_ML'] >= PNT_DET_ML_SPURIOUS, data_Xamin['EXT_LIKE'] >= EXT_LIKE_SPURIOUS)
    print(f"Nombre de sources apres Xamin: {len(data_Xamin)}")

    data_Xamin = data_Xamin[mask_spurious]
    print(f"Nombre de sources apres Xamin sans les fausses sources: {len(data_Xamin)}")

# **Filtrage du flux pour les amas et les AGN**

In [170]:
print(f"\n{'='*50}")
print("STATISTIQUES DE FILTRAGE")
print(f"Nombre initial d'amas : {len(data_input_AMAS)}")
print(f"Nombre d'amas après masque (flux_ABS > {LIM_FLUX_CLUSTER}) : {len(data_input_AMAS[data_input_AMAS['flux_ABS'] > LIM_FLUX_CLUSTER])}")
print(f"\n{'='*50}")
print("STATISTIQUES DE FILTRAGE")
print(f"Nombre initial d'AGN : {len(data_input_AGN)}")
print(f"Nombre d'AGN après masque (flux_ABS > {LIM_FLUX_AGN}) : {len(data_input_AGN[data_input_AGN['Fx_s_G14'] > LIM_FLUX_AGN])}")


mask_flux_cluster = data_input_AMAS['flux_ABS'] > LIM_FLUX_CLUSTER
data_input_AMAS = data_input_AMAS[mask_flux_cluster]

mask_flux_agn = data_input_AGN['Fx_s_G14'] > LIM_FLUX_AGN
data_input_AGN = data_input_AGN[mask_flux_agn]


STATISTIQUES DE FILTRAGE
Nombre initial d'amas : 18995
Nombre d'amas après masque (flux_ABS > 3e-15) : 942

STATISTIQUES DE FILTRAGE
Nombre initial d'AGN : 976895
Nombre d'AGN après masque (flux_ABS > 3e-15) : 15012


# **Correlation**

In [171]:
def Correlator(tableXAMIN, tableINPUT,
                           ra_xamin, dec_xamin,
                           ra_input, dec_input,
                           id_xamin, id_input,
                           search_radius):

    # ///////////////////// Corrélation /////////////////////////

    # Initialisation des tables de résultats
    unique_matches    = Table(names=(id_input, id_xamin, 'DISTANCE'), dtype=('i8', 'i8', 'f4'))
    ambiguous_matches = Table(names=(id_input,  'EXT_RATE_MOS', 'EXT_RATE_PN', id_xamin, 'DISTANCE'), dtype=('i8', 'f8', 'f8', 'i8', 'f4'))

    # Conversion en SkyCoord
    coords_bef = SkyCoord(ra=tableINPUT[ra_input]*u.deg, dec=tableINPUT[dec_input]*u.deg)
    coords_aft = SkyCoord(ra=tableXAMIN[ra_xamin]*u.deg, dec=tableXAMIN[dec_xamin]*u.deg)

    # Rayon de recherche
    search_radius_arcsec = (search_radius* u.deg).to(u.arcsec)  # en arcsec

    for i in range(len(tableXAMIN)):

        dist_arcsec = coords_aft[i].separation(coords_bef).arcsecond
        matches = np.where(dist_arcsec < search_radius_arcsec.value)[0]
        n_matches = len(matches)
        
        if n_matches == 1:
            unique_matches.add_row({
                id_xamin: tableXAMIN[id_xamin][i],
                id_input: tableINPUT[id_input][matches[0]],
                'DISTANCE': dist_arcsec[matches[0]]
            })
        elif n_matches > 1:
            for j in matches:
                ambiguous_matches.add_row({
                    id_xamin: tableXAMIN[id_xamin][i],
                    id_input: tableINPUT[id_input][j],
                    'EXT_RATE_MOS': tableXAMIN['EXT_RATE_MOS'][i],
                    'EXT_RATE_PN': tableXAMIN['EXT_RATE_PN'][i],
                    'DISTANCE': dist_arcsec[j]
                })
    print("\n" + "="*60)
    print(f"- Correspondances uniques: \033[91m{len(unique_matches)}\033[0m")
    print(f"- Nombre de sources du catalogues Xamin correlées avec ambiguités: \033[91m{len(set(ambiguous_matches[id_xamin]))}\033[0m \ncad le nombre de sources Xamin qui ont des correspondances multiples avec celles  du catalogue d'entree")
    print("="*60)

    # /////// Création de la table des matches avec distance minimale ///////////

    ambiguous_matches_min_dist = Table(names=ambiguous_matches.colnames, dtype=ambiguous_matches.dtype)

    for id in set(ambiguous_matches[id_xamin]): # Groupement par ID et recherche de la distance minimale
        
        matches_for_id = ambiguous_matches[ambiguous_matches[id_xamin] == id] # Sélection des matches pour cet ID
        idx_min = np.argmin(matches_for_id['DISTANCE']) # Trouver l'indice du match avec la distance minimale
        ambiguous_matches_min_dist.add_row(matches_for_id[idx_min]) # Ajouter ce match à la nouvelle table

    # Vérification
    n_ambiguous = len(set(ambiguous_matches[id_xamin]))
    n_selected = len(ambiguous_matches_min_dist)
    check_passed = n_ambiguous == n_selected

    print("\n" + "="*60)
    print(f"Litiges : SÉLECTION PAR DISTANCE MINIMALE")
    print(f"Nombre d'IDs ambigus uniques : {n_ambiguous}")
    print(f"Correspondances sélectionnées : {n_selected}")
    print(f"Vérification : {'✓ OK' if check_passed else '✗ ÉCHEC'} ({n_ambiguous} == {n_selected})")
    print("="*60)

    
    # ///////////////////// Concaténation /////////////////////////

    common_columns = [id_input, id_xamin, 'DISTANCE'] # Sélection des colonnes communes
    unique_subset = unique_matches[common_columns]
    ambiguous_subset = ambiguous_matches_min_dist[common_columns]
    final_catalog = vstack([unique_subset, ambiguous_subset])
    final_catalog.sort(id_input)

    print("\n" + "="*60)
    print("                     *** BILAN ***")
    print(f"Nombre de sources du catalogue d'entree: {len(tableINPUT)}")
    print(f"Nombre de correspondances uniques : {len(unique_matches)}")
    print(f"Nombre de correspondances ambigües traitées : {len(ambiguous_matches_min_dist)}")
    print(f"\033[91mTotal dans le catalogue final : {len(final_catalog)}\033[0m")
    print("="*60)

    matched_full = join(final_catalog, tableXAMIN, keys=id_xamin, join_type='inner')
    matched_full = join(matched_full, tableINPUT, keys=id_input, join_type='inner')
    print(f"\nDimensions de la table finale: {len(matched_full)} lignes x {len(matched_full.columns)} colonnes")

    return matched_full

In [172]:
data_correl_AMAS = Correlator(data_Xamin, data_input_AMAS,
                            ra_xamin = 'EXT_RA', dec_xamin = 'EXT_DEC',
                            ra_input = 'R.A.', dec_input = 'Dec',
                            id_xamin = 'ID_Xamin', id_input = 'ID',
                            search_radius = SEARCH_RADIUS_CLUSTER)


- Correspondances uniques: 496
- Nombre de sources du catalogues Xamin correlées avec ambiguités: 2 
cad le nombre de sources Xamin qui ont des correspondances multiples avec celles  du catalogue d'entree

Litiges : SÉLECTION PAR DISTANCE MINIMALE
Nombre d'IDs ambigus uniques : 2
Correspondances sélectionnées : 2
Vérification : ✓ OK (2 == 2)

                     *** BILAN ***
Nombre de sources du catalogue d'entree: 942
Nombre de correspondances uniques : 496
Nombre de correspondances ambigües traitées : 2
Total dans le catalogue final : 498

Dimensions de la table finale: 498 lignes x 128 colonnes


In [173]:
data_correl_AGN = Correlator(data_Xamin, data_input_AGN,
                                ra_xamin = 'PNT_RA', dec_xamin = 'PNT_DEC',
                                ra_input = 'ra', dec_input = 'dec',
                                id_xamin = 'ID_Xamin', id_input = 'id',
                                search_radius = SEARCH_RADIUS_AGN)


- Correspondances uniques: 7076
- Nombre de sources du catalogues Xamin correlées avec ambiguités: 82 
cad le nombre de sources Xamin qui ont des correspondances multiples avec celles  du catalogue d'entree

Litiges : SÉLECTION PAR DISTANCE MINIMALE
Nombre d'IDs ambigus uniques : 82
Correspondances sélectionnées : 82
Vérification : ✓ OK (82 == 82)

                     *** BILAN ***
Nombre de sources du catalogue d'entree: 15012
Nombre de correspondances uniques : 7076
Nombre de correspondances ambigües traitées : 82
Total dans le catalogue final : 7158

Dimensions de la table finale: 7158 lignes x 128 colonnes


# **Sauvegarde des fichiers de corrélations filtrés**

In [174]:
output_path = os.path.expanduser(f'~/Documents/TransformerProject/results/Correl_clusters_agn_10ks/Clusters_matches_r{SEARCH_RADIUS_CLUSTER*3600:.2g}arcsec_flux{LIM_FLUX_CLUSTER}_10ks.fits')
data_correl_AMAS.write(output_path, format='fits', overwrite=True)

with fits.open(output_path, mode='update') as hdul:
    hdr = hdul[1].header
    hdr['COMMENT'] = 'Catalogue complet des correspondances avant/apres Xamin'
    hdr['R_MATCH'] = (SEARCH_RADIUS_CLUSTER, 'Rayon de matching (arcsec)')
    hdr['N_MATCH'] = (len(data_correl_AMAS), 'Nombre de correspondances')

print(f"Catalogue complet sauvegardé dans : {output_path}")
print(f"Dimensions : {len(data_correl_AMAS)} lignes x {len(data_correl_AMAS.columns)} colonnes")

Catalogue complet sauvegardé dans : /local/home/sh275430/Documents/TransformerProject/results/Correl_clusters_agn_10ks/Clusters_matches_r30arcsec_flux3e-15_10ks.fits
Dimensions : 498 lignes x 128 colonnes


In [175]:
output_path = os.path.expanduser(f'~/Documents/TransformerProject/results/Correl_clusters_agn_10ks/AGN_matches_r{SEARCH_RADIUS_AGN*3600:.2g}arcsec_flux{LIM_FLUX_AGN}_10ks.fits')
data_correl_AGN.write(output_path, format='fits', overwrite=True)

with fits.open(output_path, mode='update') as hdul:
    hdr = hdul[1].header
    hdr['COMMENT'] = 'Catalogue complet des correspondances avant/apres Xamin'
    hdr['R_MATCH'] = (SEARCH_RADIUS_AGN, 'Rayon de matching (arcsec)')
    hdr['N_MATCH'] = (len(data_correl_AGN), 'Nombre de correspondances')

print(f"Catalogue complet sauvegardé dans : {output_path}")
print(f"Dimensions : {len(data_correl_AGN)} lignes x {len(data_correl_AGN.columns)} colonnes")

Catalogue complet sauvegardé dans : /local/home/sh275430/Documents/TransformerProject/results/Correl_clusters_agn_10ks/AGN_matches_r10arcsec_flux3e-15_10ks.fits
Dimensions : 7158 lignes x 128 colonnes


# **Suppression des multi-correspondances**

Rappel: correlation Output Xamin vs Input Clusters/AGN

donc plusieurs sources Xamin peuvent avoir ete correlee avec une meme source d entree! d ou la redondance des identifiants des donnees d entree

In [176]:
def stat_repet_in_data(table, key):
    ids = table[key].data
    unique_ids, counts = np.unique(ids, return_counts = True)
    stats = {
        'tot_entrees' : len(table),
        'ids_uniques' : len(unique_ids),
        'ids_dupliques' : np.sum(counts>1),
        'max_repetitions' : np.max(counts),
        'nbre_redondants' : np.sum(counts - 1)
    }
    print("-"*30)
    print(f'Total entrees dans la table: {stats['tot_entrees']}')
    print(f'Nbre d\'{key} distincts: {stats['ids_uniques']}')
    print(f'Nbre d\'{key} dupliques: {stats['ids_dupliques']}')
    print(f'Nbre max de rep: {stats['max_repetitions']}')
    print(f'Nbre de val redondantes (le nbre a suppr si plus de redondance): {stats['nbre_redondants']}')

    mask = ~np.isin(ids, unique_ids)
    #print(mask.shape)
    #print(ids.shape)
    duplicate_ids = unique_ids[counts > 1]
    mask = np.isin(ids, duplicate_ids)
    return ids[mask]

def suppr_redondance(table, key):
    table_grouped = table.group_by(key)

    min_indices = []
    for i in range(len(table_grouped.groups)):
        grp = table_grouped.groups[i]
        min_idx = np.argmin(grp['DISTANCE'])
        min_indices.append(table_grouped.groups.indices[i] + min_idx)
    table_min_per_id = table[min_indices]
    return table_min_per_id

In [177]:
id_redondants_AGN  = stat_repet_in_data(data_correl_AGN, key='id')
ID_redondants_AMAS = stat_repet_in_data(data_correl_AMAS, key='ID')

data_correl_AMAS_filt = suppr_redondance(data_correl_AMAS, key ='ID')
data_correl_AGN_filt  = suppr_redondance(data_correl_AGN, key ='id')

print("-"*30)
print(f"\nAGN: {len(data_correl_AGN)}")
print(f"AGN restants: {len(data_correl_AGN_filt)}")
print(f"\nAmas: {len(data_correl_AMAS)}")
print(f"Amas restants: {len(data_correl_AMAS_filt)}")

------------------------------
Total entrees dans la table: 7158
Nbre d'id distincts: 7154
Nbre d'id dupliques: 4
Nbre max de rep: 2
Nbre de val redondantes (le nbre a suppr si plus de redondance): 4
------------------------------
Total entrees dans la table: 498
Nbre d'ID distincts: 481
Nbre d'ID dupliques: 17
Nbre max de rep: 2
Nbre de val redondantes (le nbre a suppr si plus de redondance): 17
------------------------------

AGN: 7158
AGN restants: 7154

Amas: 498
Amas restants: 481


# **Sauvegarde des fichiers de correlations filtres**

In [178]:
output_path = os.path.expanduser(f'~/Documents/TransformerProject/results/Correl_clusters_agn_10ks/Clusters_matches_r{SEARCH_RADIUS_CLUSTER*3600:.2g}arcsec_flux{LIM_FLUX_CLUSTER}_10ks_filtered.fits')
data_correl_AMAS_filt.write(output_path, format='fits', overwrite=True)

with fits.open(output_path, mode='update') as hdul:
    hdr = hdul[1].header
    hdr['COMMENT'] = 'Catalogue complet des correspondances avant/apres Xamin'
    hdr['R_MATCH'] = (SEARCH_RADIUS_CLUSTER, 'Rayon de matching (arcsec)')
    hdr['N_MATCH'] = (len(data_correl_AMAS_filt), 'Nombre de correspondances')

print(f"Catalogue complet sauvegardé dans : {output_path}")
print(f"Dimensions : {len(data_correl_AMAS_filt)} lignes x {len(data_correl_AMAS_filt.columns)} colonnes")

Catalogue complet sauvegardé dans : /local/home/sh275430/Documents/TransformerProject/results/Correl_clusters_agn_10ks/Clusters_matches_r30arcsec_flux3e-15_10ks_filtered.fits
Dimensions : 481 lignes x 128 colonnes


In [179]:
output_path = os.path.expanduser(f'~/Documents/TransformerProject/results/Correl_clusters_agn_10ks/AGN_matches_r{SEARCH_RADIUS_AGN*3600:.2g}arcsec_flux{LIM_FLUX_AGN}_10ks_filtered.fits')
data_correl_AGN_filt.write(output_path, format='fits', overwrite=True)

with fits.open(output_path, mode='update') as hdul:
    hdr = hdul[1].header
    hdr['COMMENT'] = 'Catalogue complet des correspondances avant/apres Xamin'
    hdr['R_MATCH'] = (SEARCH_RADIUS_AGN, 'Rayon de matching (arcsec)')
    hdr['N_MATCH'] = (len(data_correl_AGN_filt), 'Nombre de correspondances')

print(f"Catalogue complet sauvegardé dans : {output_path}")
print(f"Dimensions : {len(data_correl_AGN_filt)} lignes x {len(data_correl_AGN_filt.columns)} colonnes")

Catalogue complet sauvegardé dans : /local/home/sh275430/Documents/TransformerProject/results/Correl_clusters_agn_10ks/AGN_matches_r10arcsec_flux3e-15_10ks_filtered.fits
Dimensions : 7154 lignes x 128 colonnes


# **Nombre de C1 et C2**

In [180]:
def CompteurC1C2(data, name, ext_c1_c2 = EXT_C1_C2, ext_like_c1 = EXT_LIKE_C1, ext_like_c2 = EXT_LIKE_C2):
    data_numeric = data.to_pandas()

    # Définition des classes C1 et C2
    cond_C1 = np.logical_and((data_numeric['EXT'] > ext_c1_c2) , (data_numeric['EXT_LIKE'] >= ext_like_c1))
    cond_C2 = np.logical_and(np.logical_and((data_numeric['EXT'] > ext_c1_c2) , (data_numeric['EXT_LIKE'] < ext_like_c1)),
                            (data_numeric['EXT_LIKE'] > ext_like_c2))

    n_C1 = sum(cond_C1)
    n_C2 = sum(cond_C2)
    ni_C1_ni_C2 = len(data_numeric) - (n_C1+n_C2)

    print("\n\n" + "="*70)
    print(f"Total dans le catalogue {name} : {len(data_numeric)}")
    print(f"\nNombre d'amas dans la classe C1 (EXT>{ext_c1_c2} ET EXT_LIKE>={ext_like_c1}): {n_C1}")
    print(f"Nombre d'amas dans la classe C2 (EXT>{ext_c1_c2} ET {ext_like_c2}<EXT_LIKE<{ext_like_c1}): {n_C2}")
    print(f"Nombre d'amas ni dans C1, ni dans C2: {ni_C1_ni_C2}")
    print("="*70)

In [181]:
CompteurC1C2(data_Xamin, "Xamin")
CompteurC1C2(data_correl_AMAS_filt, "AMAS")
CompteurC1C2(data_correl_AGN_filt, "AGN")



Total dans le catalogue Xamin : 17752

Nombre d'amas dans la classe C1 (EXT>5 ET EXT_LIKE>=33): 117
Nombre d'amas dans la classe C2 (EXT>5 ET 15<EXT_LIKE<33): 178
Nombre d'amas ni dans C1, ni dans C2: 17457


Total dans le catalogue AMAS : 481

Nombre d'amas dans la classe C1 (EXT>5 ET EXT_LIKE>=33): 93
Nombre d'amas dans la classe C2 (EXT>5 ET 15<EXT_LIKE<33): 67
Nombre d'amas ni dans C1, ni dans C2: 321


Total dans le catalogue AGN : 7154

Nombre d'amas dans la classe C1 (EXT>5 ET EXT_LIKE>=33): 33
Nombre d'amas dans la classe C2 (EXT>5 ET 15<EXT_LIKE<33): 58
Nombre d'amas ni dans C1, ni dans C2: 7063


In [182]:
# Convertir les colonnes ID_Xamin en tableaux numpy
ids_clusters = np.array(data_correl_AMAS_filt['ID_Xamin'])
ids_agn = np.array(data_correl_AGN_filt['ID_Xamin'])

# Trouver les intersections
common_ids = np.intersect1d(ids_clusters, ids_agn)
num_common = len(common_ids)

print(f"\nNombre de valeurs communes dans ID_Xamin : {num_common}")


Nombre de valeurs communes dans ID_Xamin : 123


In [183]:
# Création de masques booléens
mask_clusters = np.isin(data_correl_AMAS_filt['ID_Xamin'], common_ids)
mask_agn = np.isin(data_correl_AGN_filt['ID_Xamin'], common_ids)
mask_Xamin = np.isin(data_Xamin['ID_Xamin'], common_ids)

# Filtrage des tables
clusters_myst = data_correl_AMAS_filt[mask_clusters]
agn_myst = data_correl_AGN_filt[mask_agn]
Xamin_myst = data_Xamin[mask_Xamin]

# **Reconnaissance des fausses sources**: sources non correlees

In [184]:
mask_NC = ~np.isin(data_Xamin['ID_Xamin'], np.union1d(data_correl_AMAS_filt['ID_Xamin'], data_correl_AGN_filt['ID_Xamin']))
data_NC = data_Xamin[mask_NC]
print(f"=> Nombre de fausses non correlees: {len(data_NC)}")

=> Nombre de fausses non correlees: 10240


# **Sauvegarde du fichier des fausses sources**

In [185]:
output_filename = f"Spurious_10ks.fits"
output_path = os.path.expanduser(f'~/Documents/TransformerProject/results/Correl_clusters_agn_10ks/{output_filename}')
data_NC.write(output_path, format='fits', overwrite=True)

with fits.open(output_path, mode='update') as hdul:
    hdr = hdul[1].header
    hdr['COMMENT'] = 'Spurious sources (not correlated Xamin sources with the input catlog of AGNs and clusters)'

print(f"\nCatalogue complet sauvegardé dans : {output_path}")
print(f"Dimensions : {len(data_NC)} lignes x {len(data_NC.columns)} colonnes")


Catalogue complet sauvegardé dans : /local/home/sh275430/Documents/TransformerProject/results/Correl_clusters_agn_10ks/Spurious_10ks.fits
Dimensions : 10240 lignes x 114 colonnes


# **Bilan global**

In [186]:
def AffichageStatistiquesGlobales(data_xamin, data_amas, data_agn, data_nc, suppression_spurious = False):
    def suppr_spurious(data):
        mask_spurious = np.logical_or(data['PNT_DET_ML'] >= PNT_DET_ML_SPURIOUS, data['EXT_LIKE'] >= EXT_LIKE_SPURIOUS)
        data = data[mask_spurious]
        return data
    
    if(suppression_spurious):
        data_xamin = suppr_spurious(data_xamin)
        data_amas  = suppr_spurious(data_amas)
        data_agn   = suppr_spurious(data_agn)
        data_nc    = suppr_spurious(data_nc)

    # Nombre de sources Xamin a la fois correle a un amas et un AGN
    common_ids = np.intersect1d(np.array(data_amas['ID_Xamin']), np.array(data_agn['ID_Xamin']))
    num_common = len(common_ids)

    print(f"- Nombre de sources Xamin: {len(data_xamin)}")
    print(f"- Nombre d'amas: {len(data_amas) - num_common}")
    print(f"- Nombre d'AGN: {len(data_agn) - num_common}")
    print(f"- Nombre d'amas + AGN: {num_common}")
    print(f"- Nombre de fausses non correlees: {len(data_nc)}")
    print(f"\n- Nombre d'amas au total: {len(data_amas)}")
    print(f"- Nombre d'AGN au total: {len(data_agn)}")
    print(f"\nVerficication de len(data_nc) + len(data_agn) + len(data_amas) == len(data_Xamin): {len(data_nc) + len(data_agn) + len(data_amas)==len(data_xamin)}")
    print(f"Verficication de len(data_nc) + len(data_agn) + len(data_amas)- num_common == len(data_Xamin): {len(data_nc) + len(data_agn) + len(data_amas)- num_common==len(data_xamin)}")
    print(f"Verficication de len(data_nc) + len(data_agn) + len(data_amas)- num_common = {len(data_nc) + len(data_agn) + len(data_amas)- num_common}")

    total_sources = len(data_xamin)
    print("-"*30)
    print(f"\n\033[1m Amas \033[0m : {((len(data_amas)- num_common) / total_sources) * 100:>5.1f}%")
    print(f"\n\033[1m AGN \033[0m : {((len(data_agn) - num_common)/ total_sources) * 100:>5.1f}%")
    print(f"\n\033[1m Amas + AGN \033[0m : {(num_common/ total_sources) * 100:>5.1f}%")
    print(f"\n\033[1m Spurious \033[0m : {(len(data_nc) / total_sources) * 100:>5.1f}%")


In [187]:
AffichageStatistiquesGlobales(data_Xamin, data_correl_AMAS_filt, data_correl_AGN_filt, data_NC)

- Nombre de sources Xamin: 17752
- Nombre d'amas: 358
- Nombre d'AGN: 7031
- Nombre d'amas + AGN: 123
- Nombre de fausses non correlees: 10240

- Nombre d'amas au total: 481
- Nombre d'AGN au total: 7154

Verficication de len(data_nc) + len(data_agn) + len(data_amas) == len(data_Xamin): False
Verficication de len(data_nc) + len(data_agn) + len(data_amas)- num_common == len(data_Xamin): True
Verficication de len(data_nc) + len(data_agn) + len(data_amas)- num_common = 17752
------------------------------

 Amas  :   2.0%

 AGN  :  39.6%

 Amas + AGN  :   0.7%

 Spurious  :  57.7%


In [188]:
AffichageStatistiquesGlobales(data_Xamin, data_correl_AMAS_filt, data_correl_AGN_filt, data_NC, True)

- Nombre de sources Xamin: 8988
- Nombre d'amas: 185
- Nombre d'AGN: 6170
- Nombre d'amas + AGN: 111
- Nombre de fausses non correlees: 2522

- Nombre d'amas au total: 296
- Nombre d'AGN au total: 6281

Verficication de len(data_nc) + len(data_agn) + len(data_amas) == len(data_Xamin): False
Verficication de len(data_nc) + len(data_agn) + len(data_amas)- num_common == len(data_Xamin): True
Verficication de len(data_nc) + len(data_agn) + len(data_amas)- num_common = 8988
------------------------------

 Amas  :   2.1%

 AGN  :  68.6%

 Amas + AGN  :   1.2%

 Spurious  :  28.1%


# **Bilan pour les sources entre 40 et 60 photons**

In [327]:
BASE_PATH = f"~/Documents/TransformerProject/results/Correl_clusters_agn_10ks"
PATHS = {'clusters': os.path.expanduser(f"{BASE_PATH}/Clusters_matches_r{SEARCH_RADIUS_CLUSTER*3600:.0f}arcsec_flux{LIM_FLUX_CLUSTER}_10ks_filtered.fits"),
         'agn': os.path.expanduser(f"{BASE_PATH}/AGN_matches_r{SEARCH_RADIUS_AGN*3600:.0f}arcsec_flux{LIM_FLUX_AGN}_10ks_filtered.fits"),
         'NC':os.path.expanduser(f"{BASE_PATH}/Spurious_10ks.fits")}

# Définition des intervalles
RANGES = {'40-50': (40, 50),
          '50-60': (50, 60)}

def load_data(path):
    """Charge les données et calcule Ntot"""
    data = Table.read(path)
    #data['Ntot'] = data['INST0_EXP'] * data['PNT_RATE_MOS'] + data['INST1_EXP'] * data['PNT_RATE_PN']
    data['Ntot'] = data['PNT_SCTS_MOS'] + data['PNT_SCTS_PN']
    return data

# Filtrage des données
def filter_data(data_dict):
    result = {}  
    for key in data_dict:  # Parcourir chaque type de données (clusters et AGN)
        for range_name, (Nmin, Nmax) in RANGES.items(): # Parcourir chaque intervalle (40-50 et 50-60)
            mask = (data_dict[key]['Ntot'] >= Nmin) & (data_dict[key]['Ntot'] <= Nmax)
            result[f"{key}_{range_name}"] = data_dict[key][mask] # Filtrer et stocker dans le dictionnaire
    return result

In [328]:
def AffichageStatistiques(chemin, suppression_spurious = False):
    data = {key: load_data(path) for key, path in chemin.items()}
    filtered_data = filter_data(data)
    
    def suppr_spurious(data):
        mask_spurious = np.logical_or(data['PNT_DET_ML'] >= PNT_DET_ML_SPURIOUS, data['EXT_LIKE'] >= EXT_LIKE_SPURIOUS)
        data = data[mask_spurious]
        return data
    
    # Accès aux tables filtrées
    clusters_40_50 = filtered_data['clusters_40-50']
    clusters_50_60 = filtered_data['clusters_50-60']
    agn_40_50      = filtered_data['agn_40-50']
    agn_50_60      = filtered_data['agn_50-60']
    spurious_40_50 = filtered_data['NC_40-50']
    spurious_50_60 = filtered_data['NC_50-60']

    if(suppression_spurious):
        clusters_40_50 = suppr_spurious(clusters_40_50)
        clusters_50_60 = suppr_spurious(clusters_50_60)
        agn_40_50      = suppr_spurious(agn_40_50)
        agn_50_60      = suppr_spurious(agn_50_60)
        spurious_40_50 = suppr_spurious(spurious_40_50)  
        spurious_50_60 = suppr_spurious(spurious_50_60)
    
    common_ids_40_50 = np.intersect1d(np.array(clusters_40_50['ID_Xamin']), np.array(agn_40_50['ID_Xamin']))
    num_common_40_50 = len(common_ids_40_50)

    common_ids_50_60 = np.intersect1d(np.array(clusters_50_60['ID_Xamin']), np.array(agn_50_60['ID_Xamin']))
    num_common_50_60 = len(common_ids_50_60)

    print(f'Nombre de sources (40-50) : {len(clusters_40_50)+len(agn_40_50)+len(spurious_40_50)-num_common_40_50}')
    print(f'Nombre de sources (50-60) : {len(clusters_50_60)+len(agn_50_60)+len(spurious_50_60)-num_common_50_60}')

    # Pourcentage
    print("\n" + "="*60)
    print("{:^60}".format("STATISTIQUES"))
    print("="*60)
    for category in ['clusters', 'agn', 'clusters_agn', 'NC']:
        print(f"\n\033[1m{category.upper()}\033[0m")
        print("-"*30)
    
        for range_name in RANGES.keys():
            key = f"{category}_{range_name}"
            
            if(category != 'clusters_agn'):
                if(suppression_spurious):
                    n_sources = len(suppr_spurious(filtered_data[key]))
                else:
                    n_sources = len(filtered_data[key])
            if range_name == '40-50':
                if category == 'clusters' or category == 'agn':
                    n_sources = n_sources - num_common_40_50
                if category == 'clusters_agn':
                    n_sources = num_common_40_50
                total_sources = len(clusters_40_50)+len(agn_40_50)+len(spurious_40_50) - num_common_40_50
            else:
                if category == 'clusters' or category == 'agn':
                    n_sources = n_sources - num_common_50_60
                if category == 'clusters_agn':
                    n_sources = num_common_50_60
                total_sources = len(clusters_50_60)+len(agn_50_60)+len(spurious_50_60) - num_common_50_60
            global_percentage = (n_sources / total_sources) * 100
            
            print(f"• Intervalle {range_name}: {n_sources:>4} {global_percentage:>5.1f}%")

In [329]:
AffichageStatistiques(PATHS)

Nombre de sources (40-50) : 949
Nombre de sources (50-60) : 674

                        STATISTIQUES                        

CLUSTERS
------------------------------
• Intervalle 40-50:   20   2.1%
• Intervalle 50-60:    9   1.3%

AGN
------------------------------
• Intervalle 40-50:  767  80.8%
• Intervalle 50-60:  610  90.5%

CLUSTERS_AGN
------------------------------
• Intervalle 40-50:   14   1.5%
• Intervalle 50-60:   10   1.5%

NC
------------------------------
• Intervalle 40-50:  148  15.6%
• Intervalle 50-60:   45   6.7%


In [330]:
AffichageStatistiques(PATHS, True)

Nombre de sources (40-50) : 949
Nombre de sources (50-60) : 674

                        STATISTIQUES                        

CLUSTERS
------------------------------
• Intervalle 40-50:   20   2.1%
• Intervalle 50-60:    9   1.3%

AGN
------------------------------
• Intervalle 40-50:  767  80.8%
• Intervalle 50-60:  610  90.5%

CLUSTERS_AGN
------------------------------
• Intervalle 40-50:   14   1.5%
• Intervalle 50-60:   10   1.5%

NC
------------------------------
• Intervalle 40-50:  148  15.6%
• Intervalle 50-60:   45   6.7%


_____
_____

# **Overlays**

In [193]:
def plot_tile_with_sources_after_correlation(ra, dec, mr,
                                            show_Xamin_sources,
                                            show_ID_and_flux,
                                            imgs_dir = "/local/home/sh275430/Documents/TransformerProject/data/catalogs/Data_10ks/imgs", 
                                            output_dir = "~/Documents/TransformerProject/results/Correl_clusters_agn_10ks/Ambigous_images"):
    """
    Plot a specific tile image with sources overlaid after correlation analysis
    
    Parameters:
    -----------
    ra : float
        Right Ascension coordinate (e.g., 146.75)
    dec : float 
        Declination coordinate (e.g., -1.75)
    mr : bool
        True for wavelet-transformed images
    show_Xamin_sources : bool
        True for showing Xamin sources
    imgs_dir : str
        Directory containing the mosaic FITS files
    output_dir : str
        Directory to save the output plots
    """
    # Resolve paths
    imgs_dir = os.path.expanduser(imgs_dir)
    output_dir = os.path.expanduser(output_dir + f'{'/Wavelets_images' if mr else '/Photons_images'}')
    
    # Construct image filename based on mr parameter
    img_file = f"Tile-{ra}-{dec}_m12pn_b2_mosaic{'_mr' if mr else ''}.fits"
    img_path = os.path.join(imgs_dir, img_file)
    
    # Load image data
    with fits.open(img_path) as hdul:
        img_data = hdul[0].data
        header = hdul[0].header
        wcs = WCS(header)
    
    # Convert coordinates to pixels
    coords_AGN = SkyCoord(agn_myst["ra"], 
                          agn_myst["dec"], 
                          frame='icrs', 
                          unit='deg')
    x_AGN, y_AGN = wcs.world_to_pixel(coords_AGN)

    coords_AMAS = SkyCoord(clusters_myst["R.A."],
                          clusters_myst["Dec"], 
                          frame='icrs', 
                          unit='deg')
    x_AMAS, y_AMAS = wcs.world_to_pixel(coords_AMAS)

    # Create figure
    fig, ax = plt.subplots(figsize=(10, 10), subplot_kw={'projection': wcs})
    
    # Display image with proper scaling
    ax.imshow(
        img_data,
        alpha=1,
        cmap="gray",
        norm=simple_norm(img_data, "linear", max_percent=98),
        origin='lower'
    )

    # *******  Input clusters *******
    counter = 0
    for i, (x, y) in enumerate(zip(x_AMAS, y_AMAS)):
        # Check if source is within image bounds
        if (0 <= x <= img_data.shape[1] and 0 <= y <= img_data.shape[0]):
            ax.scatter(x, y,
                    s=50,
                    marker='s',
                    facecolors='none',
                    edgecolors='blue',
                    linewidths=1.8,
                    label=f"Amas" if counter == 0 else None)
            counter += 1
            
            if show_ID_and_flux:    
                # Format flux value
                flux_str = "{:.2e}".format(clusters_myst['flux_ABS'][i]).replace('e-0', 'e-').replace('e+0', 'e+')
                
                # Add annotation
                ax.annotate(
                    f"ID: {clusters_myst['ID'][i]}\nFlux: {flux_str}",
                    xy=(x +  1.2, y),
                    xytext=(5, 0),
                    textcoords='offset points',
                    color='red',
                    fontsize=6,
                    bbox=dict(facecolor='white', alpha=0.5, edgecolor='none'))

    # ******* Input AGNs *******
    counter = 0
    for x, y in zip(x_AGN, y_AGN):
        # Check if source is within image bounds
        if (0 <= x <= img_data.shape[1] and 0 <= y <= img_data.shape[0]):
            ax.scatter(x, y,
                    s=50,
                    marker='*',
                    facecolors='none',
                    edgecolors='red',
                    linewidths=1.8,
                    label=f"AGN" if counter == 0 else None)
            counter += 1
    
    if(show_Xamin_sources): # Tracer les sources après Xamin (points verts)
        coords_aft = SkyCoord(data_Xamin["PNT_RA"], data_Xamin["PNT_DEC"], frame='icrs', unit='deg')
        #coords_aft = SkyCoord(Xamin_myst["PNT_RA"], Xamin_myst["PNT_DEC"], frame='icrs', unit='deg')
        x_aft, y_aft = wcs.world_to_pixel(coords_aft)

        counter = 0
        for x, y in zip(x_aft, y_aft):
            # Check if source is within image bounds
            if (0 <= x <= img_data.shape[1] and 0 <= y <= img_data.shape[0]):
                ax.scatter(x, y,
                    s=50,
                    marker='D',
                    facecolors='none',
                    edgecolors='limegreen',
                    linewidths=0.9,
                    label=f"Xamin"if counter == 0 else None)
                counter += 1
    
    ax.set_xlim(0, img_data.shape[1])
    ax.set_ylim(0, img_data.shape[0])
    ax.set_xlabel('Right Ascension')
    ax.set_ylabel('Declination')
    ax.set_title(f"Tile {ra}° {dec}° - Source Comparison")
    ax.legend(loc='upper right')

    os.makedirs(output_dir, exist_ok=True)
    plot_name = f"Tile-{ra}-{dec}{'_mr' if mr else ''}_correlation_Rcluster{SEARCH_RADIUS_CLUSTER*3600:.2g}arcsec.png"
    plot_path = os.path.join(output_dir, plot_name)
    plt.savefig(plot_path, dpi=150, bbox_inches='tight')
    plt.close()
    print(f"Plot saved to: {plot_path}")

In [194]:
list_tile_centers = np.array([[146.75 + d_ra, 1.75 + d_dec] for d_ra in range(5) for d_dec in range(5)])

def trouver_tuile(ra, dec):
    """
    Trouve la tuile dont le centre est le plus proche des coordonnées données.
    Paramètres: ra, dec : Coordonnées en degrés
    Retourne: [ra_centre, dec_centre] : Coordonnées du centre de la tuile la plus proche
    """
    coord = SkyCoord(ra, dec, unit='deg', frame='icrs')
    tile_coords = SkyCoord(list_tile_centers[:,0], list_tile_centers[:,1], unit='deg', frame='icrs')
    distances = coord.separation(tile_coords) # Calcul des distances en une seule opération vectorisée
    i = np.argmin(distances)

    return list_tile_centers[i]

In [196]:
making_images = False
if(making_images):
    for d_ra in range(0,5):
        for d_dec in range(0,5):
            ra = 146.75 + d_ra
            dec = 1.75 + d_dec
            plot_tile_with_sources_after_correlation(ra = ra, 
                                                    dec = dec, 
                                                    mr = True,
                                                    show_Xamin_sources = True,
                                                    show_ID_and_flux = True)

In [ ]:
def plot_tverif(id, mr,
                show_Xamin_sources,
                show_SExtractor=False,
                NOMBRE_PHOTONS_MIN=40,
                output_dir="~/Documents/TransformerProject/results/Correl_clusters_agn_10ks/Ambigous_images",
                imgs_dir="/local/home/sh275430/Documents/TransformerProject/data/catalogs/Data_10ks/imgs",
                catalog_path_SExtractor='~/Documents/TransformerProject/data/catalogs/Data_10ks/SEX'):
    """
    Plot an image centered on a given source, with axes in arcminutes (ΔRA, ΔDec).
    """

    # --- Coordonnées de la source ---
    ra_Xamin, dec_Xamin = data_Xamin[data_Xamin['ID_Xamin'] == id][['PNT_RA', 'PNT_DEC']][0]
    ra_tile, dec_tile = trouver_tuile(ra_Xamin, dec_Xamin)

    # --- Fichiers ---
    imgs_dir = os.path.expanduser(imgs_dir)
    output_dir = os.path.expanduser(output_dir + ('/Wavelets_images' if mr else '/Photons_images'))
    os.makedirs(output_dir, exist_ok=True)

    img_file = f"Tile-{ra_tile}-{dec_tile}_m12pn_b2_mosaic{'_mr' if mr else ''}.fits"
    img_path = os.path.join(imgs_dir, img_file)

    # --- Lecture image FITS ---
    with fits.open(img_path) as hdul:
        img_data = hdul[0].data
        header = hdul[0].header
        wcs = WCS(header)

    # --- Centre en pixels ---
    center_pix = wcs.world_to_pixel(SkyCoord(ra_Xamin, dec_Xamin, unit='deg'))
    x_center, y_center = center_pix

    # --- Échelle en arcsec/pixel → arcmin ---
    pix_scale = np.abs(wcs.proj_plane_pixel_scales()[0].to(u.arcsec).value)
    pix_scale_arcmin = pix_scale / 60.0

    # --- Rayon = 7 arcmin ---
    radius_pix = int((4 * 60) / pix_scale)

    # --- Définir les bords du crop ---
    x_min = max(0, int(np.floor(x_center - radius_pix)))
    x_max = min(img_data.shape[1], int(np.ceil(x_center + radius_pix)))
    y_min = max(0, int(np.floor(y_center - radius_pix)))
    y_max = min(img_data.shape[0], int(np.ceil(y_center + radius_pix)))

    # --- Image crop ---
    cropped_img = img_data[y_min:y_max, x_min:x_max]
    ny, nx = cropped_img.shape

    # Décalage en arcmin de chaque pixel par rapport au centre
    x_extent = (np.array([0, nx]) - (x_center - x_min)) * pix_scale_arcmin
    y_extent = (np.array([0, ny]) - (y_center - y_min)) * pix_scale_arcmin

    # --- Axes arcmin centrés sur (0,0) ---
    x_arcmin = (np.arange(nx) - (x_center - x_min)) * pix_scale_arcmin
    y_arcmin = (np.arange(ny) - (y_center - y_min)) * pix_scale_arcmin

    #print(f"x_min={x_min}, x_max={x_max}, y_min={y_min}, y_max={y_max}")
    #print(f"cropped_img.shape = {cropped_img.shape}")
    
    # --- Figure ---
    fig, ax = plt.subplots(figsize=(10, 10))

    # --- Affichage de l'image ---
    vmin_fixed = 0.
    vmax_fixed = 0.00004
    ax.imshow(
        cropped_img,
        cmap="gray",
        vmin=vmin_fixed,
        vmax=vmax_fixed,
        origin='lower',
        extent=[x_extent[0], x_extent[1], y_extent[0], y_extent[1]]
    )

    # --- Fonction pour filtrer et afficher les sources dans le champ ---
    def plot_sources_arcmin(coords, marker, color, label, cross=False):
        delta_ra = (coords.ra.deg - ra_Xamin) * 60 * np.cos(np.deg2rad(dec_Xamin))
        delta_dec = (coords.dec.deg - dec_Xamin) * 60

        mask = (
            (delta_ra >= x_arcmin[0]) & (delta_ra <= x_arcmin[-1]) &
            (delta_dec >= y_arcmin[0]) & (delta_dec <= y_arcmin[-1])
        )

        if cross:
            ax.scatter(delta_ra[mask], delta_dec[mask],
                       marker=marker, s=550, color=color, linewidths=2, label=label)
        else:
            ax.scatter(delta_ra[mask], delta_dec[mask],
                       marker=marker, s=200, facecolors='none',
                       edgecolors=color, linewidths=2, label=label)

    # --- Affichage des sources ---
    coords_clusters = SkyCoord(data_input_AMAS["R.A."], data_input_AMAS["Dec"], unit='deg')
    plot_sources_arcmin(coords_clusters, 's', 'blue', "Clusters")

    coords_agn = SkyCoord(data_input_AGN["ra"], data_input_AGN["dec"], unit='deg')
    plot_sources_arcmin(coords_agn, '*', 'blue', "AGN")

    if show_Xamin_sources:
        coords_xamin = SkyCoord(data_Xamin["PNT_RA"], data_Xamin["PNT_DEC"], unit='deg')
        plot_sources_arcmin(coords_xamin, 's', 'limegreen', "Xamin")

        data_Xamin['Ntot'] = 2 * data_Xamin['PNT_SCTS_MOS'] + data_Xamin['PNT_SCTS_PN']
        sources_photons = data_Xamin[data_Xamin['Ntot'] > NOMBRE_PHOTONS_MIN]
        coords_photons = SkyCoord(sources_photons["PNT_RA"], sources_photons["PNT_DEC"], unit='deg')
        plot_sources_arcmin(coords_photons, '+', 'limegreen', f"Xamin > {NOMBRE_PHOTONS_MIN}ph", cross=True)

    if show_SExtractor:
        SEX_file = f"Tile-{ra_tile}-{dec_tile}_m12pn_b2_SEXtra_cat.fits"
        SEX_path = os.path.join(os.path.expanduser(catalog_path_SExtractor), SEX_file)
        data_SEX = Table.read(SEX_path)
        coords_SEX = SkyCoord(data_SEX["xpeak_world"], data_SEX["ypeak_world"], unit='deg')
        plot_sources_arcmin(coords_SEX, '^', '#E75480', "SExtractor")

    # --- Mise en forme ---
    ax.set_xlabel("ΔRA [arcmin]", fontsize=22)
    ax.set_ylabel("ΔDec [arcmin]", fontsize=22)
    ax.set_title(rf"ID {id} - Flux amas > {LIM_FLUX_CLUSTER} $\mathrm{{erg/cm^2/s}}$ - AGN > {LIM_FLUX_AGN} $\mathrm{{erg/cm^2/s}}$",
                 pad=20, fontsize=18)
    ax.legend(fontsize=15, facecolor='white')
    ax.grid(True, linestyle='--', alpha=0.5)
    ax.set_xlim(-4, 4)
    ax.set_ylim(-4, 4)
    plt.tick_params(axis='both', which='major', labelsize=18)
    
    # --- Sauvegarde ---
    plot_name = f"ID_{id}_7arcmin{'_mr' if mr else ''}.png"
    plot_path = os.path.join(output_dir, plot_name)
    plt.tight_layout()
    plt.savefig(plot_path, dpi=150, bbox_inches='tight')
    plt.close()
    print(f"Plot saved to: {plot_path}")


In [326]:
List_id_of_spurious = data_NC['ID_Xamin']

if(True):
    for id in List_id_of_spurious[25:55]:
        plot_tverif(id=id , mr = True, show_Xamin_sources = True, show_SExtractor = True, NOMBRE_PHOTONS_MIN = 40,
                    output_dir="~/Documents/TransformerProject/results/Correl_clusters_agn_10ks/Spurious")

Plot saved to: /local/home/sh275430/Documents/TransformerProject/results/Correl_clusters_agn_10ks/Spurious/Wavelets_images/ID_48_7arcmin_mr.png
Plot saved to: /local/home/sh275430/Documents/TransformerProject/results/Correl_clusters_agn_10ks/Spurious/Wavelets_images/ID_49_7arcmin_mr.png
Plot saved to: /local/home/sh275430/Documents/TransformerProject/results/Correl_clusters_agn_10ks/Spurious/Wavelets_images/ID_50_7arcmin_mr.png
Plot saved to: /local/home/sh275430/Documents/TransformerProject/results/Correl_clusters_agn_10ks/Spurious/Wavelets_images/ID_52_7arcmin_mr.png
Plot saved to: /local/home/sh275430/Documents/TransformerProject/results/Correl_clusters_agn_10ks/Spurious/Wavelets_images/ID_53_7arcmin_mr.png
Plot saved to: /local/home/sh275430/Documents/TransformerProject/results/Correl_clusters_agn_10ks/Spurious/Wavelets_images/ID_54_7arcmin_mr.png
Plot saved to: /local/home/sh275430/Documents/TransformerProject/results/Correl_clusters_agn_10ks/Spurious/Wavelets_images/ID_55_7arcmin

In [ ]:
making_images = False
if(making_images):
    for id in common_ids[:2]:
        plot_tverif(id=id , mr = True, show_Xamin_sources = True)

        cluster_info = clusters_myst[clusters_myst['ID_Xamin'] == id]
        
        if len(cluster_info) > 0:  # Check if any matches were found
            print(f"Identifiant de l'amas central : {cluster_info['ID'][0]}")
            print(f"Flux de l'amas central : {cluster_info['flux_ABS'][0]}")
        else:
            print(f"Aucun amas trouvé avec ID_Xamin = {id}")
